In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from backend.extractor.utils import parse_html, save_html
from backend.extractor.utils import clean_html, prepare_html, traverse_and_modify
from backend.extractor.task.container_extractor import ContainerExtractor
from backend.fetcher.fetcher import fetch_page
import asyncio

In [3]:
from backend.extractor.db import init_db

db = init_db()

In [4]:
url = "https://apply.workable.com/codelink/j/C19BD777BD/"
html = await fetch_page(url, static_fetch=False, scroll=False, expand=False)
soup = prepare_html(html)

2024-06-08 03:57:05,097 - utils.py: parse_html(): 79:	Parsing HTML string to BeautifulSoup object
--------
2024-06-08 03:57:05,117 - utils.py: clean_html(): 16:	Cleaning HTML, removing all path that does not contains navigable string
--------


[document] -> html -> body -> div -> div -> div -> header -> div -> h1 -> Marketing Manager
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> p -> span -> Sales & Marketing
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> p -> span -> Full time
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> p -> span -> strong -> Hybrid
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> div -> div -> span -> span -> Ho Chi Minh City,
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> div -> div -> span -> span -> Ho Chi Minh City,
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> div -> div -> span -> span -> Vietnam
[document] -> html -> body -> div -> div -> div -> div -> main -> div -> div -> button -> div -> span -> Share this job
[document] -> html -> body -> div -> div -> div -> div -> main -> div -> div -> button -> div -> svg -> desc -> p -> SVG

In [5]:
html

'<!DOCTYPE html><html lang="en" style="--color-input-default: #919bab; --color-input-default-rgb: 145,155,171; --color-input-default-darker5: #838EA0; --color-input-default-darker10: #748195; --color-input-default-darker15: #687488; --color-input-muted: #6a7381; --color-input-muted-rgb: 106,115,129; --color-input-muted-darker5: #5E6773; --color-input-muted-darker10: #535A65; --color-input-muted-darker15: #474E57; --color-input-label-optional: #6a7381; --color-input-label-optional-rgb: 106,115,129; --color-input-label-optional-darker5: #5E6773; --color-input-label-optional-darker10: #535A65; --color-input-label-optional-darker15: #474E57; --color-input-placeholder: #6a7381; --color-input-placeholder-rgb: 106,115,129; --color-input-placeholder-darker5: #5E6773; --color-input-placeholder-darker10: #535A65; --color-input-placeholder-darker15: #474E57; --color-border-default: #919bab; --color-border-default-rgb: 145,155,171; --color-border-default-darker5: #838EA0; --color-border-default-da

In [5]:
example_container = {
    "section_heading":[("Description", [""])],
    "heading": [("What You’ll Be Doing", [""])],
    "description": [("Work closely with the Marketing team and Senior Management to plan for marketing initiatives, coordinate the execution of the initiatives, and manage the marketing team around the smooth execution.", [""])],
}

In [35]:
extractor = ContainerExtractor(
    example_container,
    html,
)
await extractor.prepare_single_website_extract_template()


2024-06-08 04:20:19,260 - utils.py: parse_html(): 79:	Parsing HTML string to BeautifulSoup object
--------
2024-06-08 04:20:19,278 - utils.py: clean_html(): 16:	Cleaning HTML, removing all path that does not contains navigable string
--------
2024-06-08 04:20:19,285 - container_extractor.py: prepare_extract_item(): 49:	Get item tags
--------
2024-06-08 04:20:19,286 - extractor.py: find_string_tag(): 16:	Found string in soup at: <h2 id="job-description-title">Description</h2>
--------
2024-06-08 04:20:19,287 - container_extractor.py: prepare_extract_item(): 49:	Get item tags
--------
2024-06-08 04:20:19,288 - extractor.py: find_string_tag(): 16:	Found string in soup at: <h3>What Youâll Be Doing</h3>
--------
2024-06-08 04:20:19,289 - container_extractor.py: prepare_extract_item(): 49:	Get item tags
--------
2024-06-08 04:20:19,289 - extractor.py: find_string_tag(): 16:	Found string in soup at: <li>Work closely with the Marketing team and Senior Management to plan for marketing initiat

[document] -> html -> body -> div -> div -> div -> header -> div -> h1 -> Marketing Manager
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> p -> span -> Sales & Marketing
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> p -> span -> Full time
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> p -> span -> strong -> Hybrid
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> div -> div -> span -> span -> Ho Chi Minh City,
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> div -> div -> span -> span -> Ho Chi Minh City,
[document] -> html -> body -> div -> div -> div -> header -> div -> div -> div -> div -> span -> span -> Vietnam
[document] -> html -> body -> div -> div -> div -> div -> main -> div -> div -> button -> div -> span -> Share this job
[document] -> html -> body -> div -> div -> div -> div -> main -> div -> div -> button -> div -> svg -> desc -> p -> SVG

In [30]:
extractor.example_template["section_heading"][0].search_path

''

In [37]:
found_contents, _ = await extractor.container_extract_run_task()

2024-06-08 04:22:03,112 - container_extractor.py: prepare_extract_item(): 49:	Get item tags
--------
2024-06-08 04:22:03,126 - extractor.py: find_string_tag(): 16:	Found string in soup at: <h2 id="job-description-title">Description</h2>
--------
2024-06-08 04:22:03,150 - container_extractor.py: prepare_extract_item(): 49:	Get item tags
--------
2024-06-08 04:22:03,155 - extractor.py: find_string_tag(): 16:	Found string in soup at: <h3>What Youâll Be Doing</h3>
--------
2024-06-08 04:22:03,180 - container_extractor.py: prepare_extract_item(): 49:	Get item tags
--------
2024-06-08 04:22:03,197 - extractor.py: find_string_tag(): 16:	Found string in soup at: <li>Work closely with the Marketing team and Senior Management to plan for marketing initiatives, coordinate the execution of the initiatives, and manage the marketing team around the smooth execution.</li>
--------
2024-06-08 04:22:03,207 - container_extractor.py: prepare_extract(): 76:	Preparing extract
--------
2024-06-08 04:22:03

debug search path 
print template message  {'section_heading': [<backend.extractor.task.extract_item.ExtractItem object at 0x116fbf770>], 'heading': [<backend.extractor.task.extract_item.ExtractItem object at 0x116fbf7d0>], 'description': [<backend.extractor.task.extract_item.ExtractItem object at 0x116fbde80>]}
debug search path 
print template message  {'section_heading': [<backend.extractor.task.extract_item.ExtractItem object at 0x116fbf770>], 'heading': [<backend.extractor.task.extract_item.ExtractItem object at 0x116fbf7d0>], 'description': [<backend.extractor.task.extract_item.ExtractItem object at 0x116fbde80>]}
debug search path 
print template message  {'section_heading': [<backend.extractor.task.extract_item.ExtractItem object at 0x116fbf770>], 'heading': [<backend.extractor.task.extract_item.ExtractItem object at 0x116fbf7d0>], 'description': [<backend.extractor.task.extract_item.ExtractItem object at 0x116fbde80>]}
debug2 
debug1 h2
debug3 h2
debug2 
debug1 div > h3
debug3

In [38]:
found_contents

[{'section_heading': ' Description',
  'heading': ' What You’ll Be Doing Strategies & Initiatives Leadership',
  'description': ' Work closely with the Marketing team and Senior Management to plan for marketing initiatives, coordinate the execution of the initiatives, and manage the marketing team around the smooth execution. Manage proactive market research to identify trends, competitor analysis, and customer insights to inform marketing strategies and improve campaign performance. Develop data-driven brand & marketing plans aligned with business goals and objectives. Stay up-to-date on the latest marketing trends and technologies. Oversee all aspects of tech event booth preparation and execution,\xa0ensuring seamless delivery and stakeholder satisfaction whilst also working on ways to improve for future events for more exposure and engagement. Lead the monitoring and analysis of campaign performance, providing regular reports and insights to Management to assess the effectiveness of

In [65]:
from backend.extractor.extractor import find_string_tag


find_string_tag(extractor.soup, "£24.99")

2024-05-31 01:14:53,288 - extractor.py: find_string_tag(): 14:	Found string in soup at: None
--------


In [31]:
found_contents

[{'product': [],
  'price': ['Amazon\'s Choicefor "Decorative Home Accessories"',
   'Amazon\'s Choicefor "Decorative Home Accessories"',
   'Amazon\'s Choicefor "Decorative Home Accessories"',
   "Amazon's Choice",
   "Amazon's Choice",
   "Amazon's",
   'Choice',
   'for "Decorative Home Accessories"'],
  'rating': []},
 {'product': [],
  'price': ['Amazon\'s Choicefor "Decorative Home Accessories"',
   "Amazon's Choice",
   "Amazon's Choice",
   "Amazon's",
   'Choice',
   'for "Decorative Home Accessories"'],
  'rating': []},
 {'product': [], 'price': [''], 'rating': []},
 {'product': ['Resin Cool Dog Sculpture Bulldog Decorative Figurine Storage Tray Coin Bank Entrance Key Snack Holder Modern Art Statue Home Desktop Ornament Decoration(Black)'],
  'price': ['Resin Cool Dog Sculpture Bulldog Decorative Figurine Storage Tray Coin Bank Entrance Key Snack Holder Modern Art Statue Home Desktop Ornament Decoration(Black)',
   '4.6 out of 5 stars',
   '4.6 out of 5 stars',
   '4.6 out of

In [38]:
containers = await extractor.find_similar_containers()

2024-05-30 20:30:48,276 - container_extractor.py: find_similar_containers(): 110:	Getting list of similar containers
--------


In [41]:
containers[236]

<div class="s-product-image-container aok-relative s-text-center s-image-overlay-grey puis-image-overlay-grey s-padding-left-small s-padding-right-small puis-spacing-small s-height-equalized puis puis-v2zbxj9wv0qyon2055dia086ji7" style="padding-top: 0px !important;"><span class="rush-component" data-component-type="s-product-image" data-render-id="r37q8vah3xfh3z2bk03c30kkw6d" data-version-id="v2zbxj9wv0qyon2055dia086ji7"><a class="a-link-normal s-no-outline" href="/Carousel-Home-Gifts-Stacking-Decoration/dp/B084YZ2SQ7/ref=sr_1_16?c=ts&amp;dib=eyJ2IjoiMSJ9.6UJPju-fnbhlzsxcJI7h6ccVXSCninW7VtZi4Ht9yxEug-MLfqbhTg4cilFuaCcS4hsvM65Z7X7FQZON2z5a-43OrPqB3jEqs6IoFe4BXUl-AVk3h3Y8tAn2eSLe7oMFWQH7KC5rGKTvN5xIU9Cw78fbYiwAoCN3nyxAbvrMt2uryvDVFRpnjWttDJceGvFJI6xM1vxQq4tn6FiT3lTULT9yh3xqdOYydoLyOoBrglU7mnMJagF5qmhH_01JU8VPJOmepKbVFYWbICnNlkdi0ssVktsYrydQYdBOJ5VBjiM.1PFRfXlDZ_V0uhPX-SJ3W8ZDiwXIDvp5hmw9CEVEWQk&amp;dib_tag=se&amp;keywords=Decorative+Home+Accessories&amp;qid=1717073806&amp;s=kitchen&amp;s

In [37]:
found_contents[234]

{'product': ['Colorful Water Beads Crystal Gel Beads Water Beads 50000 Gel Soil Water Floating Candles Making, Wedding Party Decoration Relief as Vase Filler Decor Decorative Home Accessories Plants and Craft'],
 'price': ['Colorful Water Beads Crystal Gel Beads Water Beads 50000 Gel Soil Water Floating Candles Making, Wedding Party Decoration Relief as Vase Filler Decor Decorative Home Accessories Plants and Craft',
  '3.3 out of 5 stars',
  '3.3 out of 5 stars',
  '3.3 out of 5 stars',
  '26',
  '26',
  '100+ bought in past month',
  'No featured offers available',
  '£5.89',
  '(1 new offer)',
  ''],
 'rating': ['3.3 out of 5 stars']}

In [64]:
extractor.lca
selector = extractor.path_to_lca
print(selector)

div div div div div div section div div div div div div div div div article div div div


In [58]:
selector.replace(" ", " > ")
found = extractor.soup.select(selector)

In [66]:
found

[<div class="jeg_post_category"><a class="category-activities-programs" href="https://iuoss.com/category/activities-programs/">Activities - Programs</a></div>,
 <div class="jeg_post_info">
 <h2 class="jeg_post_title">
 <a href="https://iuoss.com/trien-lam-thuoc-dien-dan-cong-nghe-la-french-tech-2024/">Triển lãm thuộc Diễn đàn Công nghệ La French Tech 2024</a>
 </h2>
 <div class="jeg_post_meta"><div class="jeg_meta_date"><a href="https://iuoss.com/trien-lam-thuoc-dien-dan-cong-nghe-la-french-tech-2024/"> 27/05/2024</a></div></div>
 </div>,
 <div class="jeg_post_meta"><div class="jeg_meta_date"><a href="https://iuoss.com/trien-lam-thuoc-dien-dan-cong-nghe-la-french-tech-2024/"> 27/05/2024</a></div></div>,
 <div class="jeg_meta_date"><a href="https://iuoss.com/trien-lam-thuoc-dien-dan-cong-nghe-la-french-tech-2024/"> 27/05/2024</a></div>,
 <div class="jeg_post_category"><a class="category-thongbao" href="https://iuoss.com/category/thongbao/">Announcements</a></div>,
 <div class="jeg_post_

In [84]:
async def find_similar_containers(extractor):
    selector = extractor.path_to_lca.replace(" ", " > ")
    print(selector)
    return extractor.soup.select(selector)

In [85]:
set_up_container_task = asyncio.create_task(extractor.set_up_container_template_structure())
get_similar_containers_task = asyncio.create_task(
    find_similar_containers(extractor)
)
template_structure, similar_containers = await asyncio.gather(
set_up_container_task, get_similar_containers_task
)

2024-05-29 12:14:40,845 - container_extractor.py: set_up_container_template_structure(): 97:	Setting up template structure for container extraction
--------


<a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/">TB V/v nhắc nhở sinh viên về công tác phòng, chống tội phạm, vi phạm pháp luật, HIV/AIDS, ma túy, mại dâm trong ngành giáo dục năm 2024</a>
h3
<a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/"> 15/03/2024</a>
div
div
div > div > div > div > div > div > section > div > div > div > div > div > div > div > div > article > div


In [86]:
found_contents = []
tasks = []
for container in similar_containers:
    tasks.append(
        asyncio.create_task(
            extractor.extract_items_in_containers(
                container=container, template_structure=template_structure
            )
        )
    )
found_contents = await asyncio.gather(*tasks)

In [132]:
await extractor.prepare_extract()
unstructured_content = await extractor.extract_unstructured_text_from_containers()
unstructured_content

2024-05-29 15:07:25,804 - extractor.py: find_string_tag(): 14:	Found string in soup at: <a data-itm-added="1" data-itm-source="#vn_source=Folder-TheGioi_QuanSu&amp;vn_campaign=Stream&amp;vn_medium=Item-2&amp;vn_term=Desktop&amp;vn_thumb=1" data-medium="Item-2" data-thumb="1" href="https://vnexpress.net/cho-robot-trung-quoc-bat-nhay-ban-sung-khi-dien-tap-o-campuchia-4751792.html" title="ChÃ³ robot Trung Quá»c báº­t nháº£y, báº¯n sÃºng khi diá»n táº­p á» Campuchia">
    ChÃ³ robot Trung Quá»c báº­t nháº£y, báº¯n sÃºng khi diá»n táº­p á» Campuchia </a>
--------
2024-05-29 15:07:25,815 - extractor.py: find_string_tag(): 14:	Found string in soup at: <a data-itm-added="1" data-itm-source="#vn_source=Folder-TheGioi_QuanSu&amp;vn_campaign=Stream&amp;vn_medium=Item-2&amp;vn_term=Desktop&amp;vn_thumb=1" data-medium="Item-2" data-thumb="1" href="https://vnexpress.net/cho-robot-trung-quoc-bat-nhay-ban-sung-khi-dien-tap-o-campuchia-4751792.html" title="ChÃ³ robot Trung Quá»c báº­t nháº£y, bá

<article class="item-news thumb-left item-news-common">
<div class="thumb-art">
<a class="thumb thumb-5x3" data-itm-added="1" data-itm-source="#vn_source=Folder-TheGioi_QuanSu&amp;vn_campaign=Stream&amp;vn_medium=Item-2&amp;vn_term=Desktop&amp;vn_thumb=1" data-medium="Item-2" data-thumb="1" href="https://vnexpress.net/cho-robot-trung-quoc-bat-nhay-ban-sung-khi-dien-tap-o-campuchia-4751792.html" title="Chó robot Trung Quốc bật nhảy, bắn súng khi diễn tập ở Campuchia">
<video autoplay="" class="lazy thumb-gif error" data-ll-status="error" itemprop="contentUrl" loop="" muted="" playsinline="" poster="https://i1-vnexpress.vnecdn.net/2024/05/29/1234-1716954081-6963-1716954434.gif?w=300&amp;h=180&amp;q=100&amp;dpr=1&amp;fit=crop&amp;s=4kpuoQbTk1l9HGgfneJsGQ&amp;t=image" preload="auto" src="https://i1-vnexpress.vnecdn.net/2024/05/29/1234-1716954081-6963-1716954434.gif?w=300&amp;h=180&amp;q=100&amp;dpr=1&amp;fit=crop&amp;s=ZnNwYk5HxlLhr20XrhfXdQ&amp;t=video" style="transform: translateX(-50%);

['[if IE 9]><video style="display: none;"><![endif][if IE 9]></video><![endif]\nUkraine phóng \'tên lửa ATACMS\' tập kích sân bay Nga ở Lugansk Ukraine phóng tên lửa được cho là ATACMS vào sân bay Nga ở Lugansk, gây ra nhiều vụ nổ, có thể đã gây thiệt hại cho một radar tầm xa. ',
 '\nChó robot Trung Quốc bật nhảy, bắn súng khi diễn tập ở Campuchia Mẫu chó robot mới của Trung Quốc có thể cử động linh hoạt và được gắn súng trường để tấn công khi tham gia diễn tập chung với Campuchia.28',
 '[if IE 9]><video style="display: none;"><![endif][if IE 9]></video><![endif]\nUkraine lần đầu công bố ảnh tên lửa \'quái vật Frankenstein\' Quân đội Ukraine lần đầu công bố hình ảnh bệ phóng thời Liên Xô lắp ống phóng đạn tên lửa Mỹ để tạo thành tổ hợp phòng không kiểu "quái vật Frankenstein".10',
 '[if IE 9]><video style="display: none;"><![endif][if IE 9]></video><![endif]\nUkraine nói Nga sắp điều thêm hàng trăm nghìn quân đến biên giới Bộ trưởng Quốc phòng Ukraine cho biết nửa triệu quân Nga đang t

In [108]:
unextracted_contents = []
for container in similar_containers:
    contents = container.find_all(string=True, recursive=True)
    filtered = [string for string in contents if string.strip()]
    unextracted_contents.append(''.join(filtered))

In [122]:
structured, unstructured = await extractor.container_extract_run_task()

2024-05-29 14:53:20,509 - extractor.py: find_string_tag(): 14:	Found string in soup at: <a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/">TB V/v nháº¯c nhá» sinh viÃªn vá» cÃ´ng tÃ¡c phÃ²ng, chá»ng tá»i pháº¡m, vi pháº¡m phÃ¡p luáº­t, HIV/AIDS, ma tÃºy, máº¡i dÃ¢m trong ngÃ nh giÃ¡o dá»¥c nÄm 2024</a>
--------
2024-05-29 14:53:20,512 - extractor.py: find_string_tag(): 14:	Found string in soup at: <a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/"> 15/03/2024</a>
--------
2024-05-29 14:53:20,514 - container_extractor.py: find_container_from_examples(): 87:	Found container tag: div
                    Found path to container: div -> div -> div -> div -> div -> div -> section -> div -> div -> div -> div -> div -> div -> div -> div -> article -> div
--------
2024-05-29 

<div class="jeg_postblock_content">
<h3 class="jeg_post_title">
<a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/">TB V/v nhắc nhở sinh viên về công tác phòng, chống tội phạm, vi phạm pháp luật, HIV/AIDS, ma túy, mại dâm trong ngành giáo dục năm 2024</a>
</h3>
<div class="jeg_post_meta"><div class="jeg_meta_author"><span class="by">by</span> <a href="https://iuoss.com/author/lttmai/">lttmai</a></div><div class="jeg_meta_date"><a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-tac-phong-chong-toi-pham-vi-pham-phap-luat-hiv-aids-ma-tuy-mai-dam-trong-nganh-giao-duc-nam-2024/"> 15/03/2024</a></div></div>
<div class="jeg_post_excerpt">
<p>Hiện nay, tội phạm, vi phạm pháp luật về ma túy, mại dâm đang là vấn đề đáng lo ngại...</p>
</div>
</div>
article
div
div
div
div
div
div
div
div
section
div
div
div
div
div
div
<a href="https://iuoss.com/tb-v-v-nhac-nho-sinh-vien-ve-cong-

In [123]:
structured

[{'article': [],
  'date': ['',
   'Announcements',
   'Chương trình học bổng Vừ A Dính năm học 2023-2024',
   'vnthieu',
   '28/05/2024']},
 {'article': [], 'date': ['']},
 {'article': [],
  'date': ['Announcements',
   'Chương trình học bổng Vừ A Dính năm học 2023-2024',
   'vnthieu',
   '28/05/2024']},
 {'article': [], 'date': ['Announcements']},
 {'article': [],
  'date': ['Chương trình học bổng Vừ A Dính năm học 2023-2024',
   'vnthieu',
   '28/05/2024']},
 {'article': [], 'date': ['vnthieu', '28/05/2024']},
 {'article': [], 'date': ['vnthieu']},
 {'article': [], 'date': ['28/05/2024']},
 {'article': [],
  'date': ['',
   'Activities - Programs',
   'Triển lãm thuộc Diễn đàn Công nghệ La French Tech 2024',
   '27/05/2024']},
 {'article': [], 'date': ['']},
 {'article': [],
  'date': ['Activities - Programs',
   'Triển lãm thuộc Diễn đàn Công nghệ La French Tech 2024',
   '27/05/2024']},
 {'article': [], 'date': ['Activities - Programs']},
 {'article': [],
  'date

In [124]:
unstructured

['AnnouncementsChương trình học bổng Vừ A Dính năm học 2023-2024byvnthieu 28/05/2024',
 '',
 'AnnouncementsChương trình học bổng Vừ A Dính năm học 2023-2024byvnthieu 28/05/2024',
 'Announcements',
 'Chương trình học bổng Vừ A Dính năm học 2023-2024byvnthieu 28/05/2024',
 'byvnthieu 28/05/2024',
 'byvnthieu',
 ' 28/05/2024',
 'Activities - ProgramsTriển lãm thuộc Diễn đàn Công nghệ La French Tech 2024 27/05/2024',
 '',
 'Activities - ProgramsTriển lãm thuộc Diễn đàn Công nghệ La French Tech 2024 27/05/2024',
 'Activities - Programs',
 'Triển lãm thuộc Diễn đàn Công nghệ La French Tech 2024 27/05/2024',
 ' 27/05/2024',
 ' 27/05/2024',
 'AnnouncementsTHÔNG BÁO v/v hỗ trợ trực tuyến cho sinh viên tự chấm điểm rèn luyện 23/05/2024',
 '',
 'AnnouncementsTHÔNG BÁO v/v hỗ trợ trực tuyến cho sinh viên tự chấm điểm rèn luyện 23/05/2024',
 'Announcements',
 'THÔNG BÁO v/v hỗ trợ trực tuyến cho sinh viên tự chấm điểm rèn luyện 23/05/2024',
 ' 23/05/2024',
 ' 23/05/2024',
 'Activi

In [162]:



clean_found_contents = await filter_non_empty_dicts(found_contents)

In [163]:
clean_found_contents

[{'title': ['Hà Lan muốn cung cấp linh kiện để lắp Patriot cho Ukraine'],
  'details': ['Bộ trưởng Ollongren cho biết Hà Lan không muốn gửi một tổ hợp Patriot hoàn chỉnh cho Ukraine, thay vào đó cùng đối tác góp linh kiện lắp ráp.',
   '2']},
 {'title': ['Khoảnh khắc tên lửa đẩy Triều Tiên nổ tung trên không'],
  'details': ['Nhật, Hàn công bố cảnh quay từ mặt đất cho thấy tên lửa đẩy mang vệ tinh trinh sát quân sự Malligyong-1 của Triều Tiên phát nổ giữa không trung.',
   '']},
 {'title': ['Châu Âu thiếu thuốc nổ chế đạn pháo, tên lửa'],
  'details': ['Các nhà máy vũ khí châu Âu không đủ thuốc nổ để chế tạo đạn pháo và tên lửa, trong khi nỗ lực mở rộng nguồn cung vấp phải nhiều trở ngại.',
   '31']},
 {'title': ['Loạt vũ khí dẫn đường Mỹ bị Nga vô hiệu hóa ở Ukraine'],
  'details': ['Nhiều loại vũ khí chính xác cao Mỹ viện trợ cho Ukraine bị vô hiệu hóa, không thể đánh trúng mục tiêu do tác chiến điện tử Nga.',
   '46']},
 {'title': ['Thiết giáp Bradley cứu lính Ukraine giữa làn đạn']